<a href="https://colab.research.google.com/github/pvidya11/Deep_Learning/blob/main/Projects/09_RNN(LSTM)_NLP/Fake_News_Classifier_Using_Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
Training_file_path = "https://raw.githubusercontent.com/pvidya11/Deep_Learning/main/Projects/09_RNN(LSTM)_NLP/Sample_Data/fake-news/train.csv"
Test_file_path = "https://raw.githubusercontent.com/pvidya11/Deep_Learning/main/Projects/09_RNN(LSTM)_NLP/Sample_Data/fake-news/test.csv"

In [3]:
Train_df = pd.read_csv(Training_file_path)
Test_df = pd.read_csv(Test_file_path)

In [5]:
train_df_1 = Train_df
test_df_1 = Test_df

In [7]:
train_df_1.shape, test_df_1.shape

((20800, 5), (5200, 4))

In [8]:
train_df_1.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
test_df_1.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


<h1><b>Data Preprocessing</b></h1>

In [11]:
###Drop Nan Values
train_df_1 = train_df_1.dropna()
test_df_1 = test_df_1.dropna()

In [13]:
## Get the Independent Features
x = train_df_1.drop('label',axis=1)
## Get the Dependent features 
y = train_df_1['label']

In [14]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [16]:
x.shape, y.shape

((18285, 4), (18285,))

In [17]:
import tensorflow as tf

In [18]:
tf.__version__

'2.4.1'

In [19]:
from tensorflow.keras.layers import Embedding,LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

<h><b>Onehot Representation</b></h>

In [21]:
msg = x.copy()

In [22]:
msg['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [23]:
msg.reset_index(inplace=True)

In [24]:
import nltk 
import re 
from nltk.corpus import stopwords

In [25]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [27]:
# PorterStemmer is used to convert different form of words into it's root form
from nltk.stem.porter import PorterStemmer  
ps = PorterStemmer()
corpus = []      # collection of words
for i in range(0, len(msg)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', msg['title'][i])
    review = review.lower()
    review = review.split()

    review = [ ps.stem(word) for word in review if not word in stopwords.words('english') ]
    review = ' '.join(review)
    corpus.append(review)

In [28]:
corpus[:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [20]:
# vocabulary size
voc_size = 5000

In [30]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
onehot_repr[:5]

[[3895, 4023, 973, 3523, 2425, 4646, 2395, 1531, 1737, 4546],
 [1584, 347, 2604, 345, 3573, 3355, 102],
 [3299, 2662, 4090, 4371],
 [2699, 4309, 653, 559, 2709, 4820],
 [855, 3573, 4261, 3001, 1876, 4292, 3573, 1343, 3185, 2441]]

<h><b>Embedding Representation</b></h>

In [33]:

max_len = 0
for i in onehot_repr:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)

sent_len = max_len

47


In [34]:
embedded_docs = pad_sequences(onehot_repr, padding='pre',maxlen=sent_len)
print(embedded_docs)

[[   0    0    0 ... 1531 1737 4546]
 [   0    0    0 ... 3573 3355  102]
 [   0    0    0 ... 2662 4090 4371]
 ...
 [   0    0    0 ...  474 1497 2741]
 [   0    0    0 ... 2556 2333 4669]
 [   0    0    0 ... 3421 2613 1832]]


In [35]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 3895, 4023,  973, 3523, 2425, 4646, 2395,
       1531, 1737, 4546], dtype=int32)

<h2><b>Creating model</b></h2>

In natural language processing (NLP), Word embedding is a term used for the representation of words for text analysis, typically in the form of a real-valued vector that encodes the meaning of the word such that the words that are closer in the vector space are expected to be similar in meaning

In [37]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features,input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))

In [38]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 47, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [41]:
len(embedded_docs), y.shape

(18285, (18285,))

In [42]:
import numpy as np
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [43]:
x_final

array([[   0,    0,    0, ..., 1531, 1737, 4546],
       [   0,    0,    0, ..., 3573, 3355,  102],
       [   0,    0,    0, ..., 2662, 4090, 4371],
       ...,
       [   0,    0,    0, ...,  474, 1497, 2741],
       [   0,    0,    0, ..., 2556, 2333, 4669],
       [   0,    0,    0, ..., 3421, 2613, 1832]], dtype=int32)

In [45]:
y_final

array([1, 0, 1, ..., 0, 1, 1])

In [46]:
x_final.shape, y_final.shape

((18285, 47), (18285,))

<h1><b>Model Training</b></h>

In [47]:
# Finally Training
model.fit(x_final, y_final, validation_split=0.3, epochs=10, batch_size=64)

Epoch 1/10
200/200 [==============================] - 13s 57ms/step - loss: 0.4679 - accuracy: 0.7739 - val_loss: 0.2061 - val_accuracy: 0.9089
Epoch 2/10
200/200 [==============================] - 11s 53ms/step - loss: 0.1337 - accuracy: 0.9470 - val_loss: 0.2040 - val_accuracy: 0.9167
Epoch 3/10
200/200 [==============================] - 11s 54ms/step - loss: 0.0831 - accuracy: 0.9703 - val_loss: 0.2421 - val_accuracy: 0.9141
Epoch 4/10
200/200 [==============================] - 11s 54ms/step - loss: 0.0506 - accuracy: 0.9840 - val_loss: 0.2730 - val_accuracy: 0.9072
Epoch 5/10
200/200 [==============================] - 11s 54ms/step - loss: 0.0244 - accuracy: 0.9931 - val_loss: 0.3442 - val_accuracy: 0.9123
Epoch 6/10
200/200 [==============================] - 11s 55ms/step - loss: 0.0110 - accuracy: 0.9973 - val_loss: 0.3853 - val_accuracy: 0.9056
Epoch 7/10
200/200 [==============================] - 11s 55ms/step - loss: 0.0069 - accuracy: 0.9991 - val_loss: 0.4552 - val_accuracy:

<h1><b>Performance Metrics And Accuracy</b></h>

In [54]:
def preprocessing_func(x):
    msg = x.copy()
    msg.reset_index(inplace=True) 
    ps = PorterStemmer()
    corpus = []      # collection of words
    for i in range(0, len(msg)):
        review = re.sub('[^a-zA-Z]', ' ', msg['title'][i])
        review = review.lower().split()
        review = [ ps.stem(word) for word in review if not word in stopwords.words('english') ]
        review = ' '.join(review)
        corpus.append(review)
        
    voc_size = 5000
    onehot_repr = [one_hot(words,voc_size) for words in corpus]
    embedded_docs = pad_sequences(onehot_repr, padding='pre',maxlen=sent_len)
    x_final = np.array(embedded_docs)
    return x_final

In [55]:
x_test_final = preprocessing_func(test_df_1)

In [56]:
x_test_final

array([[   0,    0,    0, ...,  474, 1497, 2741],
       [   0,    0,    0, ..., 1696, 1231, 2372],
       [   0,    0,    0, ...,  474, 1497, 2741],
       ...,
       [   0,    0,    0, ...,  474, 1497, 2741],
       [   0,    0,    0, ...,  474, 1497, 2741],
       [   0,    0,    0, ...,  474, 1497, 2741]], dtype=int32)

In [57]:
y_pred = np.argmax(model.predict(x_test_final), axis=-1)